<a href="https://colab.research.google.com/github/esafari/random_Fruit_Flask_Ehram/blob/main/newmodel_with_low_experience_khadijeh_new_teta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gurobipy>=10 #inatallin guroby
import gurobipy as gp #creating an instanse from gurobi
params = {
"WLSACCESSID": '4ba3e808-365c-4083-8804-90dec73edd20',
"WLSSECRET": '81164514-2862-43f9-a5dd-539175b8ff65',
"LICENSEID": 2496393,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
model = gp.Model(env=env)
#setting gurobi license
import gurobipy as gp
from gurobipy import GRB
import random
import numpy as np
from itertools import chain
from itertools import accumulate
from scipy import stats

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2496393
Academic license 2496393 - for non-commercial use only - registered to kh___@ontariotechu.ca


In [ ]:
def arrival_gen(a):
    return np.random.poisson(a)
######################################################################################################
def transitions_L_to_H(y,p):
    return np.random.binomial(y, p)
######################################################################################################
def assign_treatment_plan(X_sumx,Q):
    return np.random.multinomial(abs(X_sumx),Q)
######################################################################################################
def no_show(Z_z,u):
    return np.random.binomial(Z_z,u)
######################################################################################################
def logestic_prob(w0,w,W,alpha,beta):
    if w<=w0:
        return 0
    else:
        return (w/(2*W))*(beta/(1+np.exp(-alpha*w)))
 # in future consider this for no-show

######################################################################################################
def compute_cost(s,x1,W):
  y=np.array(s[:W])
  Y=np.array(s[W])
  Z=np.array(s[W+1:W+1+R_max+R_H_max])
  Zne=np.array(s[W+1+R_max+R_H_max:])
  x=np.array(x1[:W])
  X=np.array(x1[W])
  xne=np.array(x1[W+1:])
  cc=[CP-i*mu for i in range(1,W+1)]
  return g*(c0*(Y-X))+cp*X+sum(c0*G*(y-x-xne))+cp*sum(x)+sum(cc*xne)+cp*sum(Z)+CP*sum(Zne)
########################################################################################################
def new_state(Y,y,X,x,xne,Z,z,zne,U,U_H,U_ne):
    D_L=arrival_gen(lam)
    D_H=arrival_gen(eta)
    z_next=np.zeros([len(z)])
    Z_next=np.zeros([len(Z)])
    zne_next=np.zeros([len(zne)])
    I=np.zeros(W)
    N=np.zeros([len(z)])
    N_H=np.zeros([len(Z)])
    N_ne=np.zeros([len(zne)])
    E=np.zeros([len(z)])
    E_H=np.zeros([len(Z)])
    E_ne=np.zeros([len(zne)])
    #X_next=0.0
    #x_next=np.zeros(W)
    Y_next=0.0
    y_next=np.zeros(W)
    y=np.array(y)
    x=np.array(x) #Z convert to np.array?
    for i in range(W):
        if i==0:
            y_next[i]=D_L
        else:
            I[i]= transitions_L_to_H(y[i]-x[i]-xne[i],Ow[i])
            y_next[i]=y[i-1]-x[i-1]-I[i-1]-xne[i-1]
    Y_next=Y-X+sum(I[:W])+D_H+y[W-1]-x[W-1]-xne[W-1]
    for r in range(R_max):
          N[r]=no_show(z[r],U[r])
          E[r]=transitions_L_to_H(z[r]-N[r],Q[r])
          N_ne[r]=no_show(zne[r],U_ne[r])
          E_ne[r]=transitions_L_to_H(zne[r]-N_ne[r],Q_ne[r])

    for r_H in range(R_H_max):
            N_H[r_H]=no_show(Z[r_H],U_H[r_H])
            E_H[r_H]=transitions_L_to_H(Z[r_H]-N_H[r_H],Q_H[r_H])
    z_next[0]=sum(x)
    Z_next[0]=X
    zne_next[0]=sum(xne)
    for r in range(1,R_max):
               z_next[r]=z[r-1]-N[r-1]+N[r]-E[r-1]
               zne_next[r]=zne[r-1]-N_ne[r-1]+N_ne[r]-E_ne[r-1]
    for r_H in range(1,R_H_max):
               Z_next[r_H]=Z[r_H-1]-N_H[r_H-1]+N_H[r_H]-E_H[r_H-1]
#    z=[row[:R[i]].tolist() for i , row in enumerate(z_next)]
 #   Z=[row[:R_H[i]].tolist() for i , row in enumerate(Z_next)]
  #  z = list(chain.from_iterable(z))
   # Z = list(chain.from_iterable(Z))
    s_next=[y_next.tolist(),Y_next.tolist(),z_next.tolist(),Z_next.tolist(),zne_next.tolist()]
    merged_list = []
    for item in s_next:
        if isinstance(item, list):
            merged_list.extend(item)
        else:
            merged_list.append(item)
    return merged_list
    ########################################################################################################
def multiflying_two_list(L1,L2):
  return [x * y for x, y in zip(L1, L2)]
  ##########################################################################################################
def multiflying_scalar_list(a,L):
  return [a * y for y in L]
  ##########################################################################################################
def make_list(T,R,z,teta11):
    b=[]
    bt=[]
    jj=0
    for i in range(T):
      a=[]
      at=[]
      for j in range(R[i]):
         a.append(z[jj])
         at.append(teta11[jj])
         jj=jj+1
      b.append(a)
      bt.append(at)
    return [b,bt]
    ########################################################################################################
def RLS_TD_update(J,V,teta,s,s_next,x):
  es=compute_cost(s,x,W)
  #s=np.arrary(s)
  J=J.reshape(-1, 1)
  s=s.reshape(-1, 1)
  s_next=np.array(s_next)
  s_next=s_next.reshape(-1,1)
  teta=teta.reshape(-1, 1)
  delta=s-gama*s_next #EI
  tddel=np.dot(np.transpose(delta),teta)  #EI*teta
  e=es-np.dot(np.transpose(delta),teta)
  J=gama*d*J+s
  ss=np.dot(J,np.transpose(delta))
  ss1=np.dot(np.transpose(delta),V)
  ss1=1+np.dot(ss1,J)
  ss=np.dot(ss,V)
  ss=np.dot(V,ss)
  teta=teta+np.dot(V,e*J)/ss1
  V=V-ss/ss1
  return [e,J,V,teta,es,delta,tddel]

  def Basiyan_APP_update(J,V,teta,s,s_next,x):
    es=compute_cost(s,x,W)
  #s=np.arrary(s)
    J=J.reshape(-1, 1)
    s=s.reshape(-1, 1)
    s_next=np.array(s_next)
    s_next=s_next.reshape(-1,1)
    teta=teta.reshape(-1, 1)
    delta=s-gama*s_next #EI
    tddel=np.dot(np.transpose(delta),teta)  #EI*teta
    e=es-np.dot(np.transpose(delta),teta)
    J=gama*d*J+s
    ss=np.dot(J,np.transpose(delta))
    ss1=np.dot(np.transpose(delta),V)
    ss1=1+np.dot(ss1,J)
    ss=np.dot(ss,V)
    ss=np.dot(V,ss)
    teta=teta+np.dot(V,e*J)/ss1
    V=V-ss/ss1
    return [e,J,V,teta,es,delta,tddel]
  ##########################################################################################################

def solve_integer_problem(y,M1,M2,teta):
    # Create a new Gurobi model
    n=len(y)
    model = gp.Model("integer_problem")
    model.setParam('OutputFlag', 0)
    # Create integer variables
    x= model.addVars(range(W+1), vtype=GRB.INTEGER, name="x")
    e= model.addVars(range(W), vtype=GRB.INTEGER, name="e")
    # Add constraints
    for i in range(W):
        model.addConstr(x[i]>= 0)  # Variables must be non-negative
        model.addConstr(e[i]>=0)
        model.addConstr(x[i]+e[i] <=y[i])  # Example upper bound constraint
    model.addConstr(sum(x[i] for i in range(W+1))<=M1)
    model.addConstr(sum(e[i] for i in range(W))<=M2)
    model.addConstr(x[W]>=0)
    model.addConstr(x[W]<=y[W])
    # Set objective function (minimize sum of variables)
    #[T1,T_H1]=Adjust_Treatment_session(R,R_H)
    model.setObjective(-g*c0*x[W]+cp*x[W]-sum(G[i]*c0 *(x[i]+e[i]) for i in  range(W))+sum((CP-(w+1)*mu) *e[w] for w in  range(W))+
    sum(cp*x[i] for i in  range(W))+gama1*(teta[W]*(-x[W]+sum((-x[i]-e[i])*Ow[i]for i in range(W-1))-x[W-1]-e[W-1])+
                                        sum(teta[i]*(x[i-1]+e[i-1])*(Ow[i-1]-1) for i in  range(1,W))+
                                             teta[W+R_max+1]*x[W]+teta[W+1]*sum(x[i] for i in range(W))+teta[W+R_max+R_H_max+1]*sum(e[i] for i in range(W))) , GRB.MINIMIZE)
                                                  # Optimize model
    model.optimize()
    selected1 = [x[i].x  for i in range(W+1)]
    selected2 = [e[i].x  for i in range(W)]
    return selected1+selected2
  ######################################################################################################
def Adjust_Treatment_session(R,R_H):
  R1=list(R)
  R1.insert(0,0)
  cR=list(accumulate(R1))
  T1=[W+1+cR[i] for i in range(len(R))]
  R1_H=list(R_H)
  R1_H.insert(0,0)
  cR_H1=list(accumulate(R1_H))
  T_H1=[W+1+sum(R)+cR_H1[i] for i in range(len(R_H))]
  return [T1,T_H1]
  #######################################################################################################
def Policy_Exploit(s,teta,epsilon):
  s1=s
  J=np.zeros((1+W+R_max+R_H_max+R_max))   #Z omega
  V=delta_v*np.eye((1+W+R_max+R_H_max+R_max))         #variance matrix
  z=np.zeros((1+W+R_max+R_H_max+R_max))
  teta0=np.zeros((1+W+R_max+R_H_max+R_max))+0.0001
  iter=200  #I max
  norm = np.linalg.norm(teta0)
  ii=0
  iii=0
  while True:
      ii=ii+1
      print("++++++++++++++++++++++++++++++++++++++++______________________________________++++++++++++++++++++++++++++++++++++++++++++++++++++++")
      for i in range(iter):
        ZR=max(M-sum(s[W+1:1+W+R_max+R_H_max]),0)
        ZRR=max(Me-sum(s[1+W+R_max+R_H_max:]),0)
        #[Z1,teta11]=make_list(T_H,R_H,s[W+R_max+1:],teta[W+R_max+1:])
        #[z1,teta12]=make_list(T,R,s[W+1:W+R_max+1],teta[W+1:W+R_max+1])
        ss=s
        x=solve_integer_problem(s[:W+1],ZR,ZRR,teta)
        s_next=new_state(s[W],s[:W],x[W],x[:W],x[W+1:],s[W+R_max+1:W+R_max+R_H_max+1],s[W+1:W+R_max+1],s[1+W+R_max+R_H_max:],U,U_H,U_ne)
        if i>iter-iter:
            print(f"un_used capacity:{ZR},un_used capacity_uex:{ZRR}\n current-state={s}\n y={s[:W]} Y={s[W]}  x={x[:W]} X={x[W]} e={x[W+1:]} \n  s_next={s_next}\n teta={teta}")
        [e,J,V,teta,csa,delta,tddel]=RLS_TD_update(J,V,teta,s,s_next,x)
        teta= np.squeeze(teta)
        J= np.squeeze(J)
        s2=s
        s=np.array(s_next)
        #if i>iter-3:
         #   print(f"C(s,a)={csa} s-gama*s_next={delta} tddelta={tddel} e={e} \n J={J}\n  teta={teta}")
          #iii=iii+1
      norm1 = np.linalg.norm(teta-teta0)
      dif=norm1/norm
      if dif<epsilon:
         break
      s=s1
      teta0=teta
      norm = np.linalg.norm(teta0)
  ZR=max(M-sum(s1[W+1:1+W+R_max+R_H_max]),0)
  ZRR=max(Me-sum(s1[1+W+R_max+R_H_max:]),0)
  x=solve_integer_problem(s1[:W+1],ZR,ZRR,teta)
  print(f"total number of while repetation:{ii}")
  #return [s1,x,teta,e,J,V,ii]
  return teta
############################################################################################################
def confidence_interval(data, confidence=0.95):
    n = len(data)
    mean, std_err = np.mean(data), stats.sem(data)
    margin_of_error = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    return mean - margin_of_error, mean + margin_of_error
############################################################################################################
def Average_RLSTM_TD(s,iter,epsilon):
  teta=np.zeros((1+W+R_max+R_H_max+R_max))+0.001
  teta_total=teta
  teta_total=Policy_Exploit(s,teta,epsilon)
  for i in range(iter):
    print(i,end=' ')
    teta=np.zeros((1+W+R_max+R_H_max+R_max))+0.001
    teta_total=np.vstack((teta_total,Policy_Exploit(s,teta,epsilon)))
  uper_alpha=[]
  lower_alpha=[]
  for i in range(1+W+R_max+R_H_max+R_max):
    lower, upper=confidence_interval(teta_total[:,i])
    uper_alpha.append(upper)
    lower_alpha.append(lower)
  teta_avg=np.average(teta_total,axis=0)
  return [lower_alpha,teta_avg,uper_alpha]


  #[ss,x,teta,e,J,V,ii]=Policy_Exploit(s,teta,epsilon)



In [ ]:
K_L=1                   #Relative importance of low
K_H=2                   #Relative importance of high
W=5                      #Maximum allowed waiting time for patients
G=np.arange(1,W+1)*K_L   #Accumulative priority of patients with low severity level
g=W*K_H                  #Accumulative priority of patients with highseverity level
T=2                     #The number of treatment plan types for patients with low severity levels
T_H=2
R=[3,4]
R_max=max(R)       #Number of recommended treatment sessions for patients with low severity level and from treatment plan type
R_H=[4,5]
R_H_max=max(R_H)       #Number of recommended treatment sessions for patients with high severity level and from treatment plan type
Q=[0,0,.4,.6]                    #The probability of selecting a treatment plan type t for patients with a low severity level
Q_H=[0,0,0,.3,.7]
Q_ne=[0,0,.4,.6]                  #The probability of selecting a treatment plan type  for high severity level
U=[.02,.03,0.04,.05] #change                  #Probability of missing the rth treatment session by a patient with severity level L and treatment plan type t
U_H=[.0,.01,.015,.02,.025]  #change
U_ne=[.02,.03,0.04,.05]
CP=60
mu=2
  #lamda in eligibility trace
c0=200  #postpone
cp=50 #delivery
lam=6 #low arrival rate
eta=4 #high arrival rate
alpha=.01 #logit function
beta=0.3 #logit function parameters
M=38 #total capacity
Me=15
gama=0.95 #discount factor
gama1=gama #Auxiliary variable for ignoring gama in Integer Model(Qfunction)
d=0.01 #lamda in eligibilty trace vector
delta_v=.01 #used to intitializing covariance matrix
teta=np.zeros((1+W+R_max+R_H_max+R_max))+0.001 #initial value of linear function parameters
epsilon=0.001
iter=1
Ow=[logestic_prob(W/3,w,W,alpha,beta) for w in np.arange(1,W+1)]
s=[5,3,3,5,3,5,3,4,5,3,3,2,4,4,5,2,3,2,2]
s=np.array(s)
[lower,teta_avg,upper]=Average_RLSTM_TD(s,iter,epsilon)
print("++++++++++++++++++++++++++++++++++++++++++++++Final Solution++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print(f"Lower_Teta={lower}\n teta_avg={teta_avg}\n Upper_Bound={upper}")


Streaming output truncated to the last 5000 lines.
 9256.83830131  916.29394969  809.16209451 -137.23697979 -907.05992551
 3222.07594971 3676.1207244  3465.93987797 1656.09349593 1004.23090345
  606.88609207 1009.8294954   792.34611095 1104.73920615]
un_used capacity:2.0,un_used capacity_uex:2.0
 current-state=[ 6.  9.  2.  7.  4. 14.  4.  6.  2.  0. -0.  4.  8. 11.  1.  3.  2.  3.
  5.]
 y=[6. 9. 2. 7. 4.] Y=14.0  x=[-0.0, -0.0, -0.0, -0.0, 2.0] X=-0.0 e=[-0.0, -0.0, -0.0, -0.0, 2.0] 
  s_next=[9.0, 6.0, 9.0, 1.0, 5.0, 19.0, 2.0, 4.0, 6.0, 1.0, -0.0, 0.0, 4.0, 9.0, 8.0, 2.0, 3.0, 2.0, 1.0]
 teta=[2766.36872188 6158.09336172 7818.68116207 7748.35234784 7283.56909928
 9318.02008689  924.03419791  797.36520011 -155.42467489 -948.53436861
 3237.44644142 3711.21025606 3485.93461644 1606.80359877  975.58578944
  617.32679906 1031.28199976  805.86518411 1093.98982416]
un_used capacity:4.0,un_used capacity_uex:7.0
 current-state=[ 9.  6.  9.  1.  5. 19.  2.  4.  6.  1. -0.  0.  4.  9.  8.  2.

KeyboardInterrupt: 

In [ ]:
for i in range(1):
  s1=[5,3,3,5,3,5,3,4,5,3,3,2,4,4,5,2,3,2,2]
# y=[8. 0. 0. 0. 0.] Y=7.0  x=[5.0, 0.0, 0.0, 0.0, 0.0] X=7.0 e=[3.0, 0.0, 0.0, 0.0, 0.0]
 # s_next=[6.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 0.0, 0.0, 7.0, 3.0, 7.0, 4.0, 2.0, 0.0, 3.0, 4.0, 2.0]
  teta=teta_avg
  gama1=gama
  x=solve_integer_problem(s1[:W+1],M-sum(s1[W+1:1+W+R_max+R_H_max]),Me-sum(s1[1+W+R_max+R_H_max:]),teta)
  print(f"sum_y={sum(s1[:W+1])}, un_used capacity:{M-sum(s1[W+1:W+R_max+1+R_H_max])},un_used capacity_uex:{Me-sum(s1[1+W+R_max+R_H_max:])}\n y={s1[:W]} Y={s1[W]}  x={x[:W]} X={x[W]} e={x[W+1:]} \n")

sum_y=24, un_used capacity:5,un_used capacity_uex:6
 y=[5, 3, 3, 5, 3] Y=5  x=[-0.0, -0.0, -0.0, 2.0, -0.0] X=3.0 e=[-0.0, -0.0, -0.0, 3.0, 3.0] 



In [ ]:
teta


array([ 2890.7421636 ,  5633.53286774,  8786.2804763 , 12363.30787801,
       15412.39988661, 15405.66175206,  1586.06021208,   869.28991839,
         -24.05985518, -1014.79892245,  3291.77203078,  3249.94931504,
        2270.31819521,  1297.48656849,    17.92051142,   164.61308884,
          25.4371466 ,   360.50062115,   837.36414871])